In [23]:
import re
import pandas as pd

In [5]:
data = pd.read_csv('train.csv',  index_col='pair_id')
data.head()

,name_1,name_2,is_duplicate
pair_id,,,
1,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0
2,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0
3,"Rishichem Distributors Pvt., Ltd.",Dsa,0
4,Powermax Rubber Factory,Co. One,0
5,Tress A/S,Longyou Industries Park Zhejiang,0


In [21]:
print("Total number of pairs: {}".format(data.shape[0]))
print("Number of duplicate pairs: {}".format(data.query("is_duplicate == 1").shape[0]))
print("Percent of duplicate pairs: {}%".format(round(data.query("is_duplicate == 1").shape[0] / data.shape[0] * 100, 3)))

Total number of pairs: 497819
Number of duplicate pairs: 3658
Percent of duplicate pairs: 0.735%


In [31]:
ru_en_pos_idx = []
ru_en_neg_idx = []
ru_ru_idx = []

max_len_ex = 0
min_len_ex = 1000

for index, row in data.iterrows():
    n1 = bool(re.search('[а-яА-Я]', row['name_1']))
    n2 = bool(re.search('[а-яА-Я]', row['name_2']))
    if n1 and n2:
        ru_ru_idx.append(index)
    if (n1 and not n2) or (n2 and not n1):
        if row['is_duplicate'] == 1:
            ru_en_pos_idx.append(index)
        else:
            ru_en_neg_idx.append(index)
            
    if len(row['name_1']) < min_len_ex:
        min_len_ex = len(row['name_1'])
    if len(row['name_2']) < min_len_ex:
        min_len_ex = len(row['name_2'])
        
    if len(row['name_1']) > max_len_ex:
        max_len_ex = len(row['name_1'])
    if len(row['name_2']) > max_len_ex:
        max_len_ex = len(row['name_2'])

print("Number of RU-RU pairs: {}".format(len(ru_ru_idx)))
print("Number of RU-EN positive pairs: {}".format(len(ru_en_pos_idx)))
print("Number of RU-EN negative pairs: {}".format(len(ru_en_neg_idx)) + "\n")

print("Max length: {}".format(max_len_ex))
print("Min length: {}".format(min_len_ex))

Number of RU-RU pairs: 70
Number of RU-EN positive pairs: 110
Number of RU-EN negative pairs: 2274

Max length: 141
Min length: 3


In [36]:
data.loc[ru_en_pos_idx]

,name_1,name_2,is_duplicate
pair_id,,,
5451,SOUDAL NV,"ООО ""СОУДАЛ""",1
16802,BOFER INSAAT ALUMINIUM SAN. VE TIC. LTD. STI,"ООО""БОФФЕР""",1
19576,OBO BETTERMANN PRODUKTION DEUTSCHLAND GMBH & C...,"ООО""ОБО БЕТТЕРМАНН""",1
24723,DORKEN GMBH. & CO. KG,"ООО""ДЁРКЕН""",1
25176,Polyglass America (MAPEI),ПОЛИГЛАС,1
...,...,...,...
470923,SAKHALIN ENERGY INVESTMENT CO. LTD,"ФИЛИАЛ КОМПАНИИ ""САХАЛИН ЭНЕРДЖИ ИНВЕСТМЕНТ КО...",1
473169,DOPPELMAYR SEILBAHNEN GMBH,"ООО""ДОППЕЛЬМАЙР РАША""",1
474719,ROTOFLEX AG,"АО ""РОТОФЛЕКС""",1


Идеи по нормализации текста:
    + перевести всё в lowercase
    + убрать абривиатуры, имена стран и т.д. (ООО, АО, Co. ...)
    + убрать все символы, кроме букв и пробелов(?)

Идеи по агментации(?):
    + пеервести названия на другой язык(?)
    + написать скрипт для добавления разных стоковых слов, чтобы увеличить набор